In [34]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h1 align="center">Synchrotron Tools</h1>

<h2 align="left">Select phase and EoS</h2>

In [83]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

import ipywidgets as widgets
import numpy as np
import sys

from ipywidgets import Layout, HBox, VBox
from IPython.core.display import display
from scipy.optimize import fsolve

global P_max
P_max, V0, EoS = 200, 40, 2

def phase_selector(phase):
    global K, Kp, V0, a0, b0, c0
    if phase == 2: 
        #Data from: Tange et al. (2012) 10.1029/2011JB008988 and Tange et al. (2009) 10.1029/2008JB005891
        V0, K, Kp, a0, b0, c0 = 162.373, 258.4, 4.1, 4.7769, 4.9284, 6.8972
    if phase == 3: 
        #Data from: Fiquet et al. (2002) American Mineralogist 87:1261-1265
        V0, K, Kp, a0, b0, c0 = 279.2, 108, 5, 4.6377, 4.6377, 15.007
    if phase == 4: 
        #Data from: Wang et al. (2012) 10.1029/2011JB009100
        if EoS == 2:
            V0, K, Kp, a0, b0, c0 = 46.55, 294, 4.85, 4.66, 4.66, 2.8403
        if EoS == 3:
            V0, K, Kp, a0, b0, c0 = 46.55, 292, 5.01, 4.66, 4.66, 2.8403
    K_box.value = ("K = " + str(K) + " GPa")
    Kp_box.value = ("K' = " + str(K))
    V0_box.value = ("V₀ = " + str(K) + " Å³")
    EoS_selector(EoS)
    
def EoS_selector(EoS):
    P_slider.value = 0
    V_slider.max = V0
    V_slider.value = V0
    if EoS == 2:
        V_slider.min = fsolve(lambda V,P: BM3(V,P), 1, args=(P_max), xtol=1e-5)
    if EoS == 3:
        V_slider.min = fsolve(lambda V,P: Vinet(V,P), 1, args=(P_max), xtol=1e-5)
            
def BM3(V,P):
    return P-((3/2)*K*(((V0/V)**(7/3))-((V0/V)**(5/3)))*(1-(3/4)*(4-Kp))*((V0/V)**(2/3)-1))

def Vinet(V,P):
    return P-(3*K*((1-((V/V0)**(1/3)))/((V/V0)**(2/3)))*np.exp((3/2)*(Kp-1)*(1-((V/V0)**(1/3)))))
            
dropdown1 = widgets.Dropdown(options = {'Choose': 1, 'MgSiO3 perovskite': 2, 'MgCO3 magnesite': 3, 'SiO2 stishovite': 4},description='Phase:',layout=Layout(width='47.5%'))
dropdown2 = widgets.Dropdown(options = {'Choose': 1, '3rd Order Birch-Murnaghan': 2, 'Vinet': 3},description='EoS:',layout=Layout(width='47.5%'))    

w1=widgets.interactive(phase_selector,phase=dropdown1)
w2=widgets.interactive(EoS_selector,EoS=dropdown2)

K_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
Kp_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
V0_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))

VBox([HBox([dropdown1, K_box, Kp_box, V0_box]), HBox([dropdown2])])

<h2 align="left">TOOL 1: PV calculator</h2>

In [82]:
def V_calc(P,EoS):
        if EoS == 2:
            V = fsolve(lambda V,P: BM3(V,P), 1, args=(P), xtol=1e-5)
        if EoS == 3:
            V = fsolve(lambda V,P: Vinet(V,P), 1, args=(P), xtol=1e-5)
        VV_box.value = ("V = " + str(np.around(V,3))[1:-1] + " Å³")
        Va_box.value = ("a = " + str(np.around((V/V0)**(1/3)*a0,3))[1:-1] + " Å")
        Vb_box.value = ("b = " + str(np.around((V/V0)**(1/3)*b0,3))[1:-1] + " Å")
        Vc_box.value = ("c = " + str(np.around((V/V0)**(1/3)*c0,3))[1:-1] + " Å")
              
P_slider = widgets.FloatSlider(min=0, max=200, step=0.1, value=0, description='P in GPa:',slider_color='green',layout=Layout(width='52.5%'))            
widgets.interactive(V_calc,EoS=dropdown2,P=P_slider)

#VV_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
#Va_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
#Vb_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
#Vc_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
box_titles = ['VV_box', 'Va_box','Vb_box','Vc_box']
boxes = [widgets.Text(disabled=True,color='black',layout=Layout(width='12%')) for i in box_titles]

VBox([HBox([P_slider]), HBox([VV_box, Va_box, Vb_box, Vc_box])])